<a href="https://colab.research.google.com/github/Kinle/collab_langchain/blob/main/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai google-search-results tweepy

In [ ]:
from getpass import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass()

··········


In [ ]:
os.environ['SERPAPI_API_KEY'] =getpass()

··········


In [ ]:
from requests import get
def scrape_linked_in():
  response = get('https://gist.githubusercontent.com/emarco177/0d6a3f93dd06634d95e46a2782ed7490/raw/fad4d7a87e3e934ad52ba2a968bad9eb45128665/eden-marco.json')
  data = response.json()
  data = {
      k: v
      for k, v in data.items()
      if v not in ([], "", "", None)
      and k not in ["people_also_viewed", "certifications"]
  }
  if data.get("groups"):
      for group_dict in data.get("groups"):
          group_dict.pop("profile_pic_url")

  return data




In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI

from langchain.agents import initialize_agent, Tool, AgentType




def lookup_linkedin(name: str) -> str:
    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
    template = """given the full name {name_of_person} I want you to get it me a link to their Linkedin profile page.
                          Your answer should contain only a URL"""

    tools_for_agent = [
        Tool(
            name="Crawl Google 4 linkedin profile page",
            func=get_profile_url,
            description="useful for when you need get the Linkedin Page URL",
        )
    ]

    agent = initialize_agent(
        tools=tools_for_agent,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
    )
    prompt_template = PromptTemplate(
        template=template, input_variables=["name_of_person"]
    )

    linked_profile_url = agent.run(prompt_template.format_prompt(name_of_person=name))
    return linked_profile_url

In [ ]:
from langchain.serpapi import SerpAPIWrapper

class CustomSerpAPIWrapper(SerpAPIWrapper):
  def __init__(self):
      super(CustomSerpAPIWrapper, self).__init__()

  @staticmethod
  def _process_response(res: dict) -> str:
      """Process response from SerpAPI."""
      if "error" in res.keys():
          raise ValueError(f"Got error from SerpAPI: {res['error']}")
      if "answer_box" in res.keys() and "answer" in res["answer_box"].keys():
          toret = res["answer_box"]["answer"]
      elif "answer_box" in res.keys() and "snippet" in res["answer_box"].keys():
          toret = res["answer_box"]["snippet"]
      elif (
          "answer_box" in res.keys()
          and "snippet_highlighted_words" in res["answer_box"].keys()
      ):
          toret = res["answer_box"]["snippet_highlighted_words"][0]
      elif (
          "sports_results" in res.keys()
          and "game_spotlight" in res["sports_results"].keys()
      ):
          toret = res["sports_results"]["game_spotlight"]
      elif (
          "knowledge_graph" in res.keys()
          and "description" in res["knowledge_graph"].keys()
      ):
          toret = res["knowledge_graph"]["description"]
      elif "snippet" in res["organic_results"][0].keys():
          toret = res["organic_results"][0]["link"]

      else:
          toret = "No good search result found"
      return toret



def get_profile_url(name: str):
    """Searches for Linkedin or twitter Profile Page."""
    search = CustomSerpAPIWrapper()
    res = search.run(f"{name}")
    return res

In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

information = """
    Kamal Haasan (born Parthasarathy Srinivasan on 7 November 1955)[4] is an Indian actor, filmmaker, screenwriter, playback singer, television presenter and politician who works mainly in Tamil cinema. He has also appeared in some Malayalam, Hindi, Telugu, Kannada and Bengali films. He is regarded as one of the finest actors in the Indian cinema. Haasan is also known for introducing many new technologies and cosmetics to the Indian film industry.[5][6][7] He has won numerous accolades, including Four National Film Awards, Nine Tamil Nadu State Film Awards, Four Nandi Awards, One Rashtrapati Award, Two Filmfare Awards and 17 Filmfare Awards South. He was awarded the Kalaimamani Award in 1984, the Padma Shri in 1990, the Padma Bhushan in 2014 and the Ordre des Arts et des Lettres (Chevalier) in 2016.[8]

Haasan started his career as a child artist in the 1960 Tamil-language film Kalathur Kannamma, for which he won the President's Gold Medal. His breakthrough as a lead actor came in the 1975 drama Apoorva Raagangal, directed by K. Balachander, in which he played a rebellious youth who falls in love with an older woman. He won his first National Film Award for his portrayal of a guileless school teacher who cares for a woman who suffers from retrograde amnesia in Moondram Pirai (1982). He was noted for his performances in K. Viswanath's Sagara Sangamam (1983), Swathi Muthyam (1986), Mani Ratnam's Nayakan (1987), Singeetam Srinivasa Rao's "Pushpak" (1987), Suresh Krissna 's "Sathyaa"(1988), Singeetam Srinivasa Rao 's "Apoorva Sagodharargal" (1989), Santhana Bharathi 's "Guna" (1991) and "Mahanadi" (1994). Since then he has appeared in films including Hey Ram (2000), Virumaandi (2004), Dasavathaaram (2008) in which he played ten roles, Vishwaroopam (2013) and Vikram (2022). His production company, Raaj Kamal Films International, has produced several of his films.

For his philanthropic efforts, Haasan received the first Abraham Kovoor National Award in 2004. He was project ambassador for the Hridayaragam 2010 event, which raised funds for an orphanage for HIV/AIDS-affected children. In September 2010, Haasan launched a children's cancer relief fund and gave roses to children with cancer at Sri Ramachandra University in Porur, Chennai. Haasan was nominated by the Indian Prime Minister Narendra Modi for the Swachh Bharat Mission. On 21 February 2018, Haasan formally launched his political party, Makkal Needhi Maiam (lit. People's Justice Centre).[9] He has also received the Golden Visa from United Arab Emirates.[10]
    """
summary_template = """
         given the linkedin information {information} about a person from I want you to create:
         1. a short summary
         2. two interesting facts about them
"""

summary_prompt_template = PromptTemplate(
    input_variables=["information"], template=summary_template
)

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

chain = LLMChain(llm=llm, prompt=summary_prompt_template)

# print(chain.run(information=scrape_linked_in()))
lookup_linkedin("Kinle A G")

1. Short Summary:
Eden Marco is a Customer Engineer at Google, based in Tel Aviv, Israel. He is also a best-selling instructor on Udemy, specializing in backend development. With a background in software engineering, Eden has worked at companies like Orca Security and Wizer before joining Google. He holds a Bachelor's Degree in Computer Science from Technion - Israel Institute of Technology.

2. Two Interesting Facts:
- Eden Marco has achieved significant success as an independent Udemy instructor, with two best-selling courses on the platform. His courses have attracted over 9,000 enrolled students and received more than 800 ratings, maintaining an impressive average rating of 4.7 stars.
- Prior to his career in the tech industry, Eden served as a Captain in the Israel Defense Forces. This experience likely provided him with valuable leadership skills and a strong sense of discipline that he brings to his current role as a Customer Engineer at Google.


In [ ]:
print(lookup_linkedin("Kinle A G"))



> Entering new AgentExecutor chain...
I need to find the Linkedin profile page for the person with the full name "Kinle A G".
Action: Crawl Google 4 linkedin profile page
Action Input: "Kinle A G Linkedin"
Observation: {'position': 1, 'title': 'Kinle A G - Technical Lead - Thoughtworks', 'link': 'https://in.linkedin.com/in/kinle-greshka', 'displayed_link': 'LinkedIn · 230+ followers', 'snippet': 'Kinle A G. Consultant • Technology Enthusiast • Mentor. ThoughtworksHindustan University Chennai. Coimbatore North, Tamil Nadu, India.', 'snippet_highlighted_words': ['Kinle A G'], 'about_this_result': {'keywords': ['kinle', 'ag', 'linkedin'], 'related_keywords': ['a g'], 'languages': ['English'], 'regions': ['United States']}, 'source': 'Kinle A G'}
Thought:The Linkedin profile page for "Kinle A G" is https://in.linkedin.com/in/kinle-greshka.
Final Answer: https://in.linkedin.com/in/kinle-greshka

> Finished chain.
https://in.linkedin.com/in/kinle-greshka


In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI

from langchain.agents import initialize_agent, Tool, AgentType




def lookup_twitter(name: str) -> str:
    llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
    template = """given the full name {name_of_person} I want you to get it me a link to their Twitter profile page and get username.
                          Your answer should only be the username"""

    tools_for_agent = [
        Tool(
            name="Crawl Google 4 twitter profile page",
            func=get_profile_url,
            description="useful for when you need get the Twitter Page URL",
        )
    ]

    agent = initialize_agent(
        tools=tools_for_agent,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
    )
    prompt_template = PromptTemplate(
        template=template, input_variables=["name_of_person"]
    )

    username = agent.run(prompt_template.format_prompt(name_of_person=name))
    return username

In [ ]:
lookup_twitter("Elon Musk")



> Entering new AgentExecutor chain...
I need to find the Twitter profile page of Elon Musk and extract the username.
Action: Crawl Google 4 twitter profile page
Action Input: Elon Musk
Observation: Elon Reeve Musk is a business magnate and investor. Musk is the founder, chairman, CEO and chief technology officer of SpaceX, angel investor, CEO and product architect of Tesla, Inc., owner, chairman ...
Thought:I need to refine my search to specifically find Elon Musk's Twitter profile page.
Action: Crawl Google 4 twitter profile page
Action Input: Elon Musk Twitter
Observation: Elon Musk completed his acquisition of Twitter in October 2022; Musk acted as CEO of Twitter until he stepped down in June 2023 and was replaced by Linda Yaccarino. During that timeframe, the company introduced a series of reforms and management changes.
Thought:I need to try a different search query to find Elon Musk's Twitter profile page.
Action: Crawl Google 4 twitter profile page
Action Input: Elon Musk Twit

"The username for Elon Musk's Twitter profile page is @elonmusk."